In [1]:
# Import libs
import sys
sys.path
sys.path.append('../service/')

import random
import copy

import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import gempy as gp
from gempy.utils import section_utils

#### Meta data
- AMD® Ryzen 5 3600 6-core processor × 12 
- https://www.gempy.org/documentation-1

## TODO
- function to update withouth geo_model.init_data()
- avoide double computation of solution
- deactivate regular grid
- deactivate plotting
- split into function
- check workflow
- comments
- run on GPU

#### Init geo_model

In [3]:
# instantiate the geo_model
geo_model = gp.create_model("BaseModel")

# defautl data
geo_model = gp.init_data(
    geo_model,
    extent=[0, 1000, 0, 1000, 0, 1000],
    resolution=[2, 2, 2]
)

# compile theno function
gp.set_interpolation_data(
    geo_model,
    compile_theano=True,
    theano_optimizer='fast_run',
)

Active grids: ['regular']
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                   values
range            1732.05
$C_o$            71428.6
drift equations      [3]


#### Import topological data and set topological realtionships

In [4]:
# Import topological data
series_df = pd.read_csv('./data/simple_fault_model_series.csv')
surfaces_df = pd.read_csv('./data/simple_fault_model_surfaces.csv')

In [5]:
# Set Topoligical Realtionships - Sereis
series_old = list(geo_model.series.df.to_dict()['order_series'].keys())

# add new series
for index, row in series_df.iterrows():

    serie_name = row['name']
    serie_isfault = row['isfault']
    if serie_name not in series_old:

        geo_model.add_series(series_list=[serie_name])

# remove obsolete series    
for serie in series_old:

    if serie not in series_df['name'].to_list():

        geo_model.delete_series(serie)

In [6]:
# Set Topoligical Realtionships - Surfaces
surfaces_old = geo_model.surfaces.df['surface'].to_list()

# add and update surfaces
for index, row in surfaces_df.iterrows():

    surface_name = row['name']
    surface_serie = row['serie']
    if surface_name not in surfaces_old:

        geo_model.add_surfaces(surface_list=[surface_name])
        gp.map_series_to_surfaces(
            geo_model,
            {surface_serie:surface_name}
        )

    else:

        gp.map_series_to_surfaces(
            geo_model,
            {surface_serie:surface_name}
        )


# remove obsolete surfaces
for surface in surfaces_old:

    if surface not in surfaces_df['name'].to_list():

        geo_model.delete_surfaces(surface)

#### Import and format geological imput data

In [7]:
# Import geological imput data
surface_points_input_data = pd.read_csv('./data/simple_fault_model_points.csv')
orientaions_input_data = pd.read_csv('./data/simple_fault_model_orientations.csv')

In [8]:
# Format geological_input_data
surface_points_original_df = surface_points_input_data[['X', 'Y', 'Z', 'formation']]

# rename colums
surface_points_original_df.columns = ['X', 'Y', 'Z', 'surface']

# add distribution type and parameter
surface_points_original_df['param1'] = 10

# create a deepcopy to be manipulated in place
surface_points_copy = copy.deepcopy(surface_points_original_df)

# Orientaions
orientations_df = orientaions_input_data[['X', 'Y', 'Z', 'dip', 'azimuth', 'polarity', 'formation']]

#### Section grid


In [9]:
section_dict = {'section': ([0, 0], [0, 2000], [100,100])}

In [10]:
# Collect section realizations

# constants
n_realizations = 10

# storage for section data 
list_section_data = []

# realizations
for i in range(n_realizations):
    
    # manipulate surface_points_copy in place
    surface_points_copy['X'] = ss.norm.rvs(
        loc=surface_points_original_df['X'].values,
        scale=surface_points_original_df['param1'].values)
    surface_points_copy['Y'] = ss.norm.rvs(
        loc=surface_points_original_df['Y'].values,
        scale=surface_points_original_df['param1'].values)
    surface_points_copy['Z'] = ss.norm.rvs(
        loc=surface_points_original_df['Z'].values,
        scale=surface_points_original_df['param1'].values)
    
    # Data to model
    gp.init_data(
        geo_model,
        extent=[0, 2000, 0, 2000, 0, 2000],
        resolution=[5, 5, 5],
        surface_points_df=surface_points_copy,
        orientations_df=orientations_df,
        update_surfaces=False
    )
    
    # Set fault realtions
    for index, row in series_df.iterrows():

        serie_name = row['name']
        serie_isfault = row['isfault']        
        if serie_isfault:

            geo_model.set_is_fault([serie_name])
            
    # update to interpolator
    geo_model.update_to_interpolator()
    
    # Set section grid  # Only one => client canvas
    geo_model.set_section_grid(section_dict=section_dict)
    
    # till here: until90.1 ms for 1 realizations
            
    # Compute solution
    # TODO: Fix bug!
    # till here: until 90.1 ms for 1 realizations
    # 213 m with 2x gp.compute_model()
    solution = gp.compute_model(model=geo_model)
    solution = gp.compute_model(model=geo_model)
    
    # collect extracted section data
    list_section_data.append(geo_model \
        .solutions \
        .sections[0][0] \
        .reshape(section_dict['section'][2])
    )

Active grids: ['regular']
Fault colors changed. If you do not like this behavior, set change_color to False.
Active grids: ['regular' 'sections']


/home/stefan/coding/gempy-canvas-server/env/lib/python3.6/site-packages/gempy/core/solution.py:284: UserWarning: Attribute error. Using non masked marching cubesmarching_cubes_lewiner() got an unexpected keyword argument 'mask'.
  warnings.warn('Attribute error. Using non masked marching cubes' + str(e)+'.')


Active grids: ['regular' 'sections']
Fault colors changed. If you do not like this behavior, set change_color to False.
Active grids: ['regular' 'sections']
Active grids: ['regular' 'sections']
Fault colors changed. If you do not like this behavior, set change_color to False.
Active grids: ['regular' 'sections']
Active grids: ['regular' 'sections']
Fault colors changed. If you do not like this behavior, set change_color to False.
Active grids: ['regular' 'sections']
Active grids: ['regular' 'sections']
Fault colors changed. If you do not like this behavior, set change_color to False.
Active grids: ['regular' 'sections']
Active grids: ['regular' 'sections']
Fault colors changed. If you do not like this behavior, set change_color to False.
Active grids: ['regular' 'sections']
Active grids: ['regular' 'sections']
Fault colors changed. If you do not like this behavior, set change_color to False.
Active grids: ['regular' 'sections']
Active grids: ['regular' 'sections']
Fault colors changed.

In [15]:
geo_model.set_surface_points?

In [ ]:
# Process results Stack results
section_data_stack = np.round(np.dstack(list_section_data))

# Get lithologies
lithology_ids = np.unique(section_data_stack)

In [ ]:
# Count lithology occurrences over realizations
counter_array = np.empty((
    section_dict['section'][2][0],
    section_dict['section'][2][1],
    len(lithology_ids)))

for index, lithology in enumerate(lithology_ids):
    
    counter_array[:,:,index] = np.sum((
        section_data_stack == lithology).astype(int), axis=2)

In [ ]:
probability_array = counter_array / n_realizations
entropy_map = ss.entropy(probability_array, axis=2)
plt.imshow(entropy_map, cmap='viridis')
plt.colorbar()
plt.show()